### Tests different moving target and strategies

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from numpy.linalg import norm
import statsmodels.api as sm
from scipy.spatial.distance import cosine, euclidean
from scipy.special import rel_entr

In [2]:
df = pd.read_csv('final_data.csv')
df

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,delta_IndStruct_avg,delta_BizDesc_avg,delta_RiskFirm_avg,delta_RiskGeneric_avg,delta_Obj_avg,delta_Subj_avg,delta_Fwd_avg,delta_Hist_avg,delta_analyst_avg,delta_Sentiment_avg
0,f_Aa1vUFfy,AES CORP,US00130H1059,2023-03-01,data/reports/AES_CORP/f_Aa1vUFfy_file.pdf,data/reports/AES_CORP/f_Aa1vUFfy_pages.txt,data/reports/AES_CORP/f_Aa1vUFfy_blocks.txt,2023-03,2023-03-31,00130H10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f_5OBOii0M,AES CORP,US00130H1059,2023-05-04,data/reports/AES_CORP/f_5OBOii0M_file.pdf,data/reports/AES_CORP/f_5OBOii0M_pages.txt,data/reports/AES_CORP/f_5OBOii0M_blocks.txt,2023-05,2023-05-31,00130H10,...,-0.013889,-0.117063,0.0,0.0,-0.078704,0.078704,-0.029101,0.023148,-0.009259,0.0
2,f_xQH93a0P,AES CORP,US00130H1059,2023-08-03,data/reports/AES_CORP/f_xQH93a0P_file.pdf,data/reports/AES_CORP/f_xQH93a0P_pages.txt,data/reports/AES_CORP/f_xQH93a0P_blocks.txt,2023-08,2023-08-31,00130H10,...,0.000000,0.015455,0.0,0.0,0.146199,-0.146199,-0.053258,0.000000,0.000000,0.0
3,f_ioDdiVH2,AES CORP,US00130H1059,2023-11-02,data/reports/AES_CORP/f_ioDdiVH2_file.pdf,data/reports/AES_CORP/f_ioDdiVH2_pages.txt,data/reports/AES_CORP/f_ioDdiVH2_blocks.txt,2023-11,2023-11-30,00130H10,...,0.000000,-0.002998,0.0,0.0,-0.051965,0.051965,0.016489,-0.012658,0.000000,0.0
4,f_3UcWcgnDz3SRMAFFYqWxjB,AES CORP,US00130H1059,2024-02-26,data/reports/AES_CORP/f_3UcWcgnDz3SRMAFFYqWxjB...,data/reports/AES_CORP/f_3UcWcgnDz3SRMAFFYqWxjB...,data/reports/AES_CORP/f_3UcWcgnDz3SRMAFFYqWxjB...,2024-02,2024-02-29,00130H10,...,0.009662,0.093133,0.0,0.0,-0.070079,0.070079,0.108543,-0.006665,0.014493,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4186,o_04SGcWBpglGMPqR3sPmjzB,ZOETIS INC,US98978V1035,2024-08-06,data/reports/ZOETIS_INC/o_04SGcWBpglGMPqR3sPmj...,data/reports/ZOETIS_INC/o_04SGcWBpglGMPqR3sPmj...,data/reports/ZOETIS_INC/o_04SGcWBpglGMPqR3sPmj...,2024-08,2024-08-30,98978V10,...,0.000000,-0.288889,0.0,0.0,-0.400000,0.400000,0.407843,0.000000,-0.022222,0.0
4187,o_dmtqfZ8ADLGCDsotk7icYy,ZOETIS INC,US98978V1035,2024-09-04,data/reports/ZOETIS_INC/o_dmtqfZ8ADLGCDsotk7ic...,data/reports/ZOETIS_INC/o_dmtqfZ8ADLGCDsotk7ic...,data/reports/ZOETIS_INC/o_dmtqfZ8ADLGCDsotk7ic...,2024-09,2024-09-30,98978V10,...,0.076923,0.000000,0.0,0.0,0.000000,0.000000,0.058824,0.000000,0.000000,0.0
4188,f_frs85kiCZmvfQLo7bs3fjH,ZOETIS INC,US98978V1035,2024-11-04,data/reports/ZOETIS_INC/f_frs85kiCZmvfQLo7bs3f...,data/reports/ZOETIS_INC/f_frs85kiCZmvfQLo7bs3f...,data/reports/ZOETIS_INC/f_frs85kiCZmvfQLo7bs3f...,2024-11,2024-11-29,98978V10,...,-0.076923,0.260870,0.0,0.0,0.434783,-0.434783,-0.500000,0.000000,0.000000,0.0
4189,o_bAec9s5gPlI8P7axYeLS88,ZOETIS INC,US98978V1035,2024-11-04,data/reports/ZOETIS_INC/o_bAec9s5gPlI8P7axYeLS...,data/reports/ZOETIS_INC/o_bAec9s5gPlI8P7axYeLS...,data/reports/ZOETIS_INC/o_bAec9s5gPlI8P7axYeLS...,2024-11,2024-11-29,98978V10,...,0.000000,-0.260870,0.0,0.0,-0.379227,0.379227,0.500000,0.000000,0.000000,0.0


In [3]:
df.columns

Index(['report_id', 'company_name', 'isin', 'reported_at', 'pdf_path',
       'pages_path', 'blocks_path', 'year_month', 'DATE', 'SYMBOL',
       'COMPANY_NAME', 'TICKER', 'SECTOR', 'MOMENTUM', 'VALUE', 'SIZE', 'BETA',
       'SEDOL', 'ISIN', 'RETURNS', 'report_type', 'Sales', 'Expense', 'Profit',
       'Ops', 'Liq', 'Inv', 'Fin', 'Lit', 'Emp', 'Tax', 'Acct', 'Prod',
       'Buyer', 'Process', 'Loc', 'Promo', 'Supplier', 'Mgmt', 'Strategy',
       'IndStruct', 'BizDesc', 'RiskFirm', 'RiskGeneric', 'Obj', 'Subj', 'Fwd',
       'Hist', 'analyst', 'Sentiment', 'row_count', 'date', 'entry_date',
       'entry_price', 'return_after', 'return_before', 'delta_Sales_avg',
       'delta_Expense_avg', 'delta_Profit_avg', 'delta_Ops_avg',
       'delta_Liq_avg', 'delta_Inv_avg', 'delta_Fin_avg', 'delta_Lit_avg',
       'delta_Emp_avg', 'delta_Tax_avg', 'delta_Acct_avg', 'delta_Prod_avg',
       'delta_Buyer_avg', 'delta_Process_avg', 'delta_Loc_avg',
       'delta_Promo_avg', 'delta_Supplier_avg

### Bang-Bang Evaluation: Single-Variable Max Delta Predictor

In [4]:
# --- evaluation helper (as in your code) ---
def evaluate_predictions(y_true, y_pred, method=""):
    mask = (y_true != 0) & (y_pred != 0)
    
    # Debug dropped rows
    dropped = np.where(~mask)[0]
    if len(dropped) > 0:
        print(f"[DEBUG] {method}: Dropped {len(dropped)} rows because preds or labels were 0. Examples: {dropped[:10]}")

    y_true, y_pred = y_true[mask], y_pred[mask]

    mask_up = y_true > 0
    mask_down = y_true < 0

    acc_up = (y_true[mask_up] == y_pred[mask_up]).mean() if mask_up.any() else np.nan
    acc_down = (y_true[mask_down] == y_pred[mask_down]).mean() if mask_down.any() else np.nan
    return acc_up, acc_down, len(y_true)

# --- Bang-bang max delta only ---
def evaluate_bangbang(df, feature_cols, return_col="RETURNS"):
    y_true = np.sign(df[return_col].to_numpy())

    if feature_cols:
        X = df[feature_cols].fillna(0).to_numpy()
        idx = np.abs(X).argmax(axis=1)   # index of max absolute delta
        row_choice = X[np.arange(len(X)), idx]
        y_pred = np.sign(row_choice)
    else:
        y_pred = np.zeros(len(df))

    return evaluate_predictions(y_true, y_pred, method="Bang-bang max delta")


feature_cols = [c for c in df.columns if c.startswith("delta_") and c.endswith("_avg")]

# Drop rows with NaNs or all-zero deltas
df_clean = df.dropna(subset=feature_cols)
mask_all_zero = (df_clean[feature_cols] == 0).all(axis=1)
print(f"[DEBUG] Removing {mask_all_zero.sum()} rows with all-zero deltas")
df_clean = df_clean[~mask_all_zero]

# Run evaluation
acc_up, acc_down, nrows = evaluate_bangbang(df_clean, feature_cols, return_col="return_after")
print(f"Bang-bang max delta -> acc_up: {acc_up:.3f}, acc_down: {acc_down:.3f}, rows: {nrows}")

[DEBUG] Removing 4 rows with all-zero deltas
[DEBUG] Bang-bang max delta: Dropped 553 rows because preds or labels were 0. Examples: [ 7  8 12 16 17 20 24 28 31 32]
Bang-bang max delta -> acc_up: 0.540, acc_down: 0.510, rows: 3208


In [5]:
# Filter only delta columns
delta_cols = [c for c in df.columns if c.startswith("delta_") and c.endswith("_avg")]

### Distribution of Narrative Shifts

In [6]:

# count non-zero deltas per row
df["num_shifts"] = (df[delta_cols] != 0).sum(axis=1)
avg_shifts = df["num_shifts"].mean()
print(f"Average shifts per firm-quarter: {avg_shifts:.2f}")

# split financial vs non-financial categories
fin_delta = [c for c in delta_cols if any(x in c for x in ["Sales","Expense","Profit","Ops","Liq","Inv","Fin","Lit","Emp","Tax","Acct"])]
biz_delta = [c for c in delta_cols if any(x in c for x in ["Prod","Buyer","Process","Loc","Promo","Supplier","Mgmt","Strategy","IndStruct","BizDesc"])]

df["fin_shift"] = (df[fin_delta].abs().sum(axis=1) > 0).astype(int)
df["biz_shift"] = (df[biz_delta].abs().sum(axis=1) > 0).astype(int)

print("Percent with major financial shift:", df["fin_shift"].mean())
print("Percent with major business shift:", df["biz_shift"].mean())

Average shifts per firm-quarter: 20.50
Percent with major financial shift: 0.896683369124314
Percent with major business shift: 0.8823669768551659


### Variables correlation with returns

In [7]:
# --- Correlation with return_after ---
corr_after = df[delta_cols].corrwith(df["return_after"]).sort_values(key=abs, ascending=False)

print("Top 20 correlations with return_after:")
print(corr_after.head(20))

# --- Correlation with return_before ---
corr_before = df[delta_cols].corrwith(df["return_before"]).sort_values(key=abs, ascending=False)

print("\nTop 20 correlations with return_before:")
print(corr_before.head(20))

Top 20 correlations with return_after:
delta_RiskGeneric_avg    0.143102
delta_BizDesc_avg       -0.041978
delta_Hist_avg          -0.037216
delta_Supplier_avg      -0.036183
delta_Emp_avg           -0.035802
delta_Lit_avg           -0.032933
delta_Expense_avg       -0.032319
delta_Tax_avg           -0.031124
delta_Inv_avg           -0.030962
delta_Mgmt_avg          -0.030841
delta_Fin_avg           -0.029316
delta_Prod_avg          -0.028983
delta_Acct_avg          -0.027121
delta_IndStruct_avg      0.025371
delta_Sales_avg         -0.024766
delta_Liq_avg           -0.024457
delta_Ops_avg           -0.019580
delta_Profit_avg        -0.019032
delta_Obj_avg           -0.012378
delta_Subj_avg           0.012378
dtype: float64

Top 20 correlations with return_before:
delta_RiskGeneric_avg    0.145950
delta_BizDesc_avg       -0.042755
delta_Hist_avg          -0.037570
delta_Supplier_avg      -0.036579
delta_Emp_avg           -0.035496
delta_Lit_avg           -0.032787
delta_Expense_avg    

In [8]:
# Risk Language
risk_cols = ["RiskFirm", "RiskGeneric"]
risk_freq = df[risk_cols].mean()
print("Average share of sentences with risks:\n", risk_freq)

# correlate firm-specific risk with return volatility
volatility = df.groupby("ISIN")["RETURNS"].std()
risk_corr = df.groupby("ISIN")["RiskFirm"].mean().corr(volatility)
print("Correlation (firm-specific risk vs vol):", risk_corr)

Average share of sentences with risks:
 RiskFirm       0.999940
RiskGeneric    0.980827
dtype: float64
Correlation (firm-specific risk vs vol): 0.034575415629701185


### LLM is poor at identifying Analyst Interaction

In [9]:
#Analyst Interaction
analyst_ratio = df["analyst"].mean()
print(f"Analyst-originated sentences share: {analyst_ratio:.2%}")

# check link with underperformance
analyst_perf = df.groupby("analyst")["RETURNS"].mean()
print("Avg returns conditional on analyst intervention:\n", analyst_perf)

Analyst-originated sentences share: 1.59%
Avg returns conditional on analyst intervention:
 analyst
0.000000    0.011200
0.004785   -0.009913
0.004854   -0.059374
0.006061    0.065700
0.006452   -0.062500
              ...   
0.187919    0.216784
0.193548   -0.008490
0.195122   -0.022467
0.200000    0.049164
0.500000    0.052051
Name: RETURNS, Length: 520, dtype: float64


### Tone - timeframe and sentence structure

In [10]:
# Share of forward vs historical
fwd_share = df["Fwd"].sum() / (df["Fwd"].sum() + df["Hist"].sum())
print(f"Forward-looking share: {fwd_share:.2%}")
print(f"Historical share: {1 - fwd_share:.2%}")

# Relationship with returns: correlation
corr_fwd = df["Fwd"].corr(df["return_after"])
corr_hist = df["Hist"].corr(df["return_after"])
print(f"Correlation (Fwd vs return_after): {corr_fwd:.3f}")
print(f"Correlation (Hist vs return_after): {corr_hist:.3f}")

# Group returns by forward/high vs low
df["fwd_ratio"] = df["Fwd"] / (df["Fwd"] + df["Hist"] + 1e-9)
high_fwd = df[df["fwd_ratio"] > df["fwd_ratio"].median()]["return_after"].mean()
low_fwd = df[df["fwd_ratio"] <= df["fwd_ratio"].median()]["return_after"].mean()
print(f"Avg return_after (high forward share): {high_fwd:.3%}")
print(f"Avg return_after (low forward share): {low_fwd:.3%}")


Forward-looking share: 37.92%
Historical share: 62.08%
Correlation (Fwd vs return_after): 0.018
Correlation (Hist vs return_after): -0.042
Avg return_after (high forward share): 30.397%
Avg return_after (low forward share): 7.067%


In [11]:
# Share of subjective vs objective
subj_share = df["Subj"].sum() / (df["Subj"].sum() + df["Obj"].sum())
print(f"Subjective share: {subj_share:.2%}")
print(f"Objective share: {1 - subj_share:.2%}")

# Relationship with returns
corr_subj = df["Subj"].corr(df["return_after"])
corr_obj = df["Obj"].corr(df["return_after"])
print(f"Correlation (Subj vs return_after): {corr_subj:.3f}")
print(f"Correlation (Obj vs return_after): {corr_obj:.3f}")

# High vs low subjectivity
df["subj_ratio"] = df["Subj"] / (df["Subj"] + df["Obj"] + 1e-9)
high_subj = df[df["subj_ratio"] > df["subj_ratio"].median()]["return_after"].mean()
low_subj = df[df["subj_ratio"] <= df["subj_ratio"].median()]["return_after"].mean()
print(f"Avg return_after (high subjectivity): {high_subj:.3%}")
print(f"Avg return_after (low subjectivity): {low_subj:.3%}")


Subjective share: 67.38%
Objective share: 32.62%
Correlation (Subj vs return_after): 0.019
Correlation (Obj vs return_after): -0.019
Avg return_after (high subjectivity): 29.602%
Avg return_after (low subjectivity): 7.714%


In [12]:
# realized volatility after reports
vol = df.groupby("ISIN")["return_after"].std()
subj_mean = df.groupby("ISIN")["subj_ratio"].mean()
corr_vol = subj_mean.corr(vol)
print(f"Correlation (subjectivity vs volatility): {corr_vol:.3f}")


Correlation (subjectivity vs volatility): 0.091


In [13]:
q75 = df["subj_ratio"].quantile(0.75)
q25 = df["subj_ratio"].quantile(0.25)

high_q = df[df["subj_ratio"] >= q75]["return_after"].mean()
low_q = df[df["subj_ratio"] <= q25]["return_after"].mean()

print(f"Avg return_after (top 25% subjective): {high_q:.3%}")
print(f"Avg return_after (bottom 25% subjective): {low_q:.3%}")


Avg return_after (top 25% subjective): 50.786%
Avg return_after (bottom 25% subjective): 9.396%


In [14]:
# Define objective ratio
df["obj_ratio"] = df["Obj"] / (df["Subj"] + df["Obj"] + 1e-9)

# Quartiles for objective
q75 = df["obj_ratio"].quantile(0.75)
q25 = df["obj_ratio"].quantile(0.25)

high_q = df[df["obj_ratio"] >= q75]["return_after"].mean()
low_q = df[df["obj_ratio"] <= q25]["return_after"].mean()

print(f"Avg return_after (top 25% objective): {high_q:.3%}")
print(f"Avg return_after (bottom 25% objective): {low_q:.3%}")


Avg return_after (top 25% objective): 9.396%
Avg return_after (bottom 25% objective): 50.786%


### Different moving average testing

**MTS_roll (10-10% FLIPPED)**  
n=8, mean=-8.3724, stdev=23.5717, sharpe=-0.36, hit_rate=0.25, cum_return=-50.23  
Alpha (annualized, bps): 13634.0 | t(alpha)=2.01 | p=0.045 | R²=1.00, Adj R²=1.00  
Betas: MKT=-3.766, MOMENTUM=-10.472, VALUE=7.437, SIZE=15.579  

**MTS_ewma (10-10% FLIPPED)**  
n=8, mean=-8.3640, stdev=23.6175, sharpe=-0.35, hit_rate=0.38, cum_return=-60.55  
Alpha (annualized, bps): 12447.0 | t(alpha)=1.84 | p=0.066 | R²=1.00, Adj R²=1.00  
Betas: MKT=-3.873, MOMENTUM=-9.067, VALUE=8.775, SIZE=15.466  

**MTS_median (10-10% FLIPPED)**  
n=8, mean=-8.3510, stdev=23.6014, sharpe=-0.35, hit_rate=0.38, cum_return=-63.32  
Alpha (annualized, bps): 11173.9 | t(alpha)=1.70 | p=0.089 | R²=1.00, Adj R²=1.00  
Betas: MKT=-3.417, MOMENTUM=-9.312, VALUE=8.715, SIZE=15.391  

**MTS_w_var (10-10% FLIPPED)**  
n=8, mean=-0.0092, stdev=0.0591, sharpe=-0.16, hit_rate=0.38, cum_return=-0.08  
Alpha (annualized, bps): 685.2 | t(alpha)=0.92 | p=0.356 | R²=0.32, Adj R²=-0.58  
Betas: MKT=0.076, MOMENTUM=-0.598, VALUE=-0.621, SIZE=0.142  

**MTS_w_freq (10-10% FLIPPED)**  
n=8, mean=-0.0572, stdev=0.2001, sharpe=-0.29, hit_rate=0.62, cum_return=-0.51  
Alpha (annualized, bps): 497.2 | t(alpha)=0.15 | p=0.878 | R²=0.83, Adj R²=0.61  
Betas: MKT=1.716, MOMENTUM=-5.453, VALUE=-4.687, SIZE=0.442  

**MTS_pca (10-10% FLIPPED)**  
n=8, mean=-0.0132, stdev=0.0864, sharpe=-0.15, hit_rate=0.50, cum_return=-0.13  
Alpha (annualized, bps): -2156.0 | t(alpha)=-1.20 | p=0.231 | R²=0.75, Adj R²=0.42  
Betas: MKT=1.502, MOMENTUM=-2.164, VALUE=-1.710, SIZE=0.243  

**MTS_lasso (10-10% FLIPPED)**  
n=7, mean=-0.0199, stdev=0.0795, sharpe=-0.25, hit_rate=0.43, cum_return=-0.15  
Alpha (annualized, bps): -295.6 | t(alpha)=-0.27 | p=0.790 | R²=0.86, Adj R²=0.57  
Betas: MKT=-0.273, MOMENTUM=-0.441, VALUE=1.071, SIZE=1.363  

**MTS_kl (10-10% FLIPPED)**  
n=8, mean=0.0269, stdev=0.0586, sharpe=0.46, hit_rate=0.75, cum_return=0.22  
Alpha (annualized, bps): 829.3 | t(alpha)=1.13 | p=0.260 | R²=0.47, Adj R²=-0.23  
Betas: MKT=0.770, MOMENTUM=-1.180, VALUE=-0.997, SIZE=0.216  

**MTS_js (10-10% FLIPPED)**  
n=8, mean=-0.0052, stdev=0.0630, sharpe=-0.08, hit_rate=0.50, cum_return=-0.05  
Alpha (annualized, bps): 1051.1 | t(alpha)=1.61 | p=0.106 | R²=0.47, Adj R²=-0.24  
Betas: MKT=0.122, MOMENTUM=-1.076, VALUE=-1.001, SIZE=0.097  

**MTS_cosine (10-10% FLIPPED)**  
n=8, mean=0.1158, stdev=0.2155, sharpe=0.54, hit_rate=0.75, cum_return=1.15  
Alpha (annualized, bps): -2229.2 | t(alpha)=-1.11 | p=0.267 | R²=0.63, Adj R²=0.15  
Betas: MKT=0.353, MOMENTUM=4.229, VALUE=3.792, SIZE=-0.004  

**MTS_euclid (10-10% FLIPPED)**  
n=8, mean=-0.1213, stdev=0.1862, sharpe=-0.65, hit_rate=0.00, cum_return=-0.71  
Alpha (annualized, bps): 1801.4 | t(alpha)=1.11 | p=0.266 | R²=0.79, Adj R²=0.52  
Betas: MKT=-0.514, MOMENTUM=-3.889, VALUE=-3.515, SIZE=-0.043  

**MTS_ham (10-10% FLIPPED)**  
n=8, mean=-0.8650, stdev=2.1440, sharpe=-0.40, hit_rate=0.00, cum_return=-3.06  
Alpha (annualized, bps): 2284.7 | t(alpha)=1.39 | p=0.163 | R²=1.00, Adj R²=0.99  
Betas: MKT=-0.965, MOMENTUM=-3.518, VALUE=-1.590, SIZE=1.269  

**MTS_costate (10-10% FLIPPED)**  
n=8, mean=-0.0726, stdev=0.0811, sharpe=-0.90, hit_rate=0.12, cum_return=-0.47  
Alpha (annualized, bps): 309.7 | t(alpha)=0.82 | p=0.413 | R²=0.84, Adj R²=0.64  
Betas: MKT=-0.894, MOMENTUM=-0.900, VALUE=-0.814, SIZE=-0.268  


In [21]:
# ---------- Helpers ----------
def _ensure_dt(df):
    d = df.copy()
    d["date"] = pd.to_datetime(d["date"]).dt.tz_localize(None)
    return d.sort_values(["date","ISIN"])

def _list_delta_cols(df):
    return [c for c in df.columns if c.startswith("delta_")]

def _cs_zscore(s, by):
    return s.groupby(by).transform(lambda x: (x - x.mean()) / (x.std(ddof=0)+1e-12))

def _dates_lookback(dates, i, window):
    start = max(0, i - window)
    return dates[start:i]   # strictly past


# ---------- Basic MTS ----------
def mts_rolling(df, delta_cols, window=3):
    d = _ensure_dt(df)
    raw = (d.groupby("ISIN")[delta_cols]
             .rolling(window, min_periods=1).mean()
             .reset_index(level=0, drop=True)
             .mean(axis=1))
    return _cs_zscore(raw, by=d["date"])

def mts_ewma(df, delta_cols, halflife=3):
    d = _ensure_dt(df)
    raw = (d.groupby("ISIN")[delta_cols]
             .apply(lambda g: g.ewm(halflife=halflife, adjust=False).mean().mean(axis=1))
             .reset_index(level=0, drop=True))
    return _cs_zscore(raw, by=d["date"])

def mts_median(df, delta_cols, window=3):
    d = _ensure_dt(df)
    raw = (d.groupby("ISIN")[delta_cols]
             .rolling(window, min_periods=1).median()
             .reset_index(level=0, drop=True)
             .median(axis=1))
    return _cs_zscore(raw, by=d["date"])


# ---------- Weighted ----------
def _weights_from_past(past_df, delta_cols, method="var"):
    X = past_df[delta_cols].fillna(0).to_numpy()
    if X.shape[0]==0: return None
    if method=="var":
        w = 1.0 / (np.var(X, axis=0)+1e-9)
    elif method=="freq":
        w = (X!=0).mean(axis=0)
    else:
        return None
    s = np.sum(np.abs(w))
    return w/s if s>0 else None

def mts_weighted(df, delta_cols, method="var", lookback=6, min_obs=60):
    d = _ensure_dt(df); dates = d["date"].drop_duplicates().sort_values().to_list()
    out = pd.Series(index=d.index, dtype=float)
    for i, dt in enumerate(dates):
        past_dates = _dates_lookback(dates, i, lookback)
        past = d[d["date"].isin(past_dates)]
        if len(past)<min_obs: continue
        w = _weights_from_past(past, delta_cols, method=method)
        if w is None: continue
        today = d["date"]==dt
        out.loc[today] = d.loc[today, delta_cols].fillna(0).dot(w)
    return _cs_zscore(out, by=d["date"])


# ---------- PCA & Lasso ----------
def mts_pca(df, delta_cols, n_comp=1, lookback=6, min_obs=60):
    d = _ensure_dt(df); dates = d["date"].drop_duplicates().to_list()
    out = pd.Series(index=d.index, dtype=float)
    for i, dt in enumerate(dates):
        past = d[d["date"].isin(_dates_lookback(dates, i, lookback))]
        if len(past)<min_obs: continue
        try:
            pca = PCA(n_components=n_comp).fit(past[delta_cols].fillna(0))
        except: continue
        Xt = d.loc[d["date"]==dt, delta_cols].fillna(0).to_numpy()
        out.loc[d["date"]==dt] = pca.transform(Xt)[:,0]
    return _cs_zscore(out, by=d["date"])

def mts_lasso(df, delta_cols, lookback=6, min_obs=60):
    d = _ensure_dt(df); dates = d["date"].drop_duplicates().to_list()
    out = pd.Series(index=d.index, dtype=float)
    for i, dt in enumerate(dates):
        past = d[d["date"].isin(_dates_lookback(dates, i, lookback))]
        if len(past)<min_obs: continue
        Xp = past[delta_cols].fillna(0).to_numpy()
        yp = np.arange(len(Xp))  # dummy trend target (since no RETURNS allowed)
        try:
            model = LassoCV(cv=3, n_alphas=30).fit(Xp, yp)
        except: continue
        coef = model.coef_
        if np.allclose(coef,0): continue
        Xt = d.loc[d["date"]==dt, delta_cols].fillna(0).to_numpy()
        out.loc[d["date"]==dt] = Xt @ coef
    return _cs_zscore(out, by=d["date"])


# ---------- KL/JS/Cosine/Euclid/Hamilton/Costate ----------
def mts_kl_js_cosine(df, delta_cols, lookback=None, min_obs=None):
    d = _ensure_dt(df); delta = d[delta_cols].fillna(0).to_numpy()
    KL, JS, COS, EUC, HAM, COST = [], [], [], [], [], []
    for i in range(len(d)):
        if i==0:
            KL.append(np.nan); JS.append(np.nan); COS.append(np.nan)
            EUC.append(np.nan); HAM.append(np.nan); COST.append(np.nan)
            continue
        p, q = delta[i], delta[i-1]
        p_pos = np.abs(p)+1e-9; q_pos = np.abs(q)+1e-9
        p_norm, q_norm = p_pos/p_pos.sum(), q_pos/q_pos.sum()
        kl = np.sum(rel_entr(p_norm, q_norm))
        m = 0.5*(p_norm+q_norm)
        js = 0.5*(np.sum(rel_entr(p_norm,m))+np.sum(rel_entr(q_norm,m)))
        cos = 1 - cosine(p,q) if np.any(q) else np.nan
        euc = euclidean(p,q)
        ham = np.sum(p/(np.var(p)+1e-6))
        cost = np.sum(np.abs(p-q))
        KL.append(kl); JS.append(js); COS.append(cos); EUC.append(euc); HAM.append(ham); COST.append(cost)
    d["MTS_kl"]      = _cs_zscore(pd.Series(KL, index=d.index), by=d["date"])
    d["MTS_js"]      = _cs_zscore(pd.Series(JS, index=d.index), by=d["date"])
    d["MTS_cosine"]  = _cs_zscore(pd.Series(COS, index=d.index), by=d["date"])
    d["MTS_euclid"]  = _cs_zscore(pd.Series(EUC, index=d.index), by=d["date"])
    d["MTS_ham"]     = _cs_zscore(pd.Series(HAM, index=d.index), by=d["date"])
    d["MTS_costate"] = _cs_zscore(pd.Series(COST, index=d.index), by=d["date"])
    return d


In [22]:
def build_all_mts(df, lookback=6, min_obs=60):
    d = _ensure_dt(df)
    delta_cols = _list_delta_cols(d)

    d["MTS_roll"]   = mts_rolling(d, delta_cols)
    d["MTS_ewma"]   = mts_ewma(d, delta_cols)
    d["MTS_median"] = mts_median(d, delta_cols)

    d["MTS_w_var"]  = mts_weighted(d, delta_cols, method="var", lookback=lookback, min_obs=min_obs)
    d["MTS_w_freq"] = mts_weighted(d, delta_cols, method="freq", lookback=lookback, min_obs=min_obs)

    d["MTS_pca"]    = mts_pca(d, delta_cols, 1, lookback, min_obs)
    d["MTS_lasso"]  = mts_lasso(d, delta_cols, lookback, min_obs)

    d = mts_kl_js_cosine(d, delta_cols)

    return d

In [24]:
# -----------------------------
# Simple performance line (like your printout)
# -----------------------------
def perf_line(port_df):
    r = port_df["port_ret"].dropna().astype(float)
    n = len(r)
    if n==0:
        return "n=0, mean=nan, stdev=nan, sharpe=nan, hit_rate=nan, cum_return=nan"
    mu, sd = r.mean(), r.std(ddof=1)
    sharpe = mu/sd if sd>0 else np.nan
    hit = (r>0).mean()
    cumr = (1+r).prod()-1
    return f"n={n}, mean={mu:.4f}, stdev={sd:.4f}, sharpe={sharpe:.2f}, hit_rate={hit:.2f}, cum_return={cumr:.2f}"

# -----------------------------
# Factor regression (NW-HAC SE)
# -----------------------------
def factor_regression_alpha_clean(port_df, factors_df, hac_lags=1, ann_factor=4):
    import statsmodels.api as sm
    f = factors_df.copy()
    f["date"] = pd.to_datetime(f["date"]).dt.tz_localize(None)
    p = port_df.copy()
    p["date"] = pd.to_datetime(p["date"]).dt.tz_localize(None)

    fac_cols = [c for c in f.columns if c!="date"]
    merged = pd.merge(p[["date","port_ret"]], f[["date"]+fac_cols], on="date", how="inner").dropna()

    if merged.empty or len(merged)<3:
        return {"n": len(merged), "alpha_q": np.nan, "alpha_ann_bps": np.nan,
                "t_alpha": np.nan, "p_alpha": np.nan, "r2": np.nan, "r2_adj": np.nan,
                "beta": {c: np.nan for c in fac_cols}, "merged": merged}

    y = merged["port_ret"].astype(float).values
    X = sm.add_constant(merged[fac_cols].astype(float).values)
    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})

    alpha = res.params[0]
    alpha_se = np.sqrt(res.cov_params()[0,0]) if np.isfinite(res.cov_params()[0,0]) else np.nan
    t_alpha = alpha/alpha_se if (alpha_se and alpha_se>0) else np.nan
    p_alpha = res.pvalues[0] if 0 in range(len(res.pvalues)) else np.nan
    alpha_ann_bps = alpha * ann_factor * 1e4

    betas = dict(zip(fac_cols, res.params[1:] if len(res.params)>1 else []))
    return {
        "n": int(res.nobs),
        "alpha_q": alpha,
        "alpha_ann_bps": alpha_ann_bps,
        "t_alpha": t_alpha,
        "p_alpha": p_alpha,
        "r2": res.rsquared,
        "r2_adj": res.rsquared_adj,
        "beta": betas,
        "res": res,
        "merged": merged
    }

# -----------------------------
# Run for ALL MTS (10–10 flipped) and print
# -----------------------------
def run_all_mts_factor_eval(df_mts,
                            mts_cols,
                            ret_col="return_after",
                            flip=True,
                            quantile=0.10,
                            hac_lags=1,
                            factors_df=None,
                            xs_factor_cols=("MOMENTUM","VALUE","SIZE")):
    # build factors if not provided
    if factors_df is None:
        factors_df = build_quarterly_factors_from_xs(df_mts, ret_col=ret_col,
                                                     use_cols=xs_factor_cols, qcut=0.3)

    for mts in mts_cols:
        try:
            port, _ = ls_portfolio_quarterly(df_mts, mts_col=mts, ret_col=ret_col,
                                             quantile=quantile, print_debug=False)
            if flip:
                port["port_ret"] = -port["port_ret"]
        except Exception as e:
            print(f"\n{mts} (10-10% {'FLIPPED' if flip else 'NORMAL'})")
            print("Error building portfolio:", e)
            continue

        print(f"\n{mts} (10-10% {'FLIPPED' if flip else 'NORMAL'})")
        print(perf_line(port))

        reg = factor_regression_alpha_clean(port, factors_df, hac_lags=hac_lags, ann_factor=4)
        alpha_bps = reg["alpha_ann_bps"]
        t_a, p_a  = reg["t_alpha"], reg["p_alpha"]
        r2, r2a   = reg["r2"], reg["r2_adj"]
        beta_str = ", ".join([f"{k}={v:.3f}" for k,v in reg["beta"].items()]) if reg["beta"] else "—"

        print(f"Alpha (annualized, bps): {alpha_bps if np.isfinite(alpha_bps) else np.nan:.1f} | "
              f"t(alpha)={t_a if np.isfinite(t_a) else np.nan:.2f} | "
              f"p={p_a if np.isfinite(p_a) else np.nan:.3f} | "
              f"R²={r2 if np.isfinite(r2) else np.nan:.2f}, Adj R²={r2a if np.isfinite(r2a) else np.nan:.2f}")
        print(f"Betas: {beta_str}")


In [27]:
# -----------------------------
# Factor builders (quarterly EW long-short)
# -----------------------------
def _q_end(s):
    s = pd.to_datetime(s).dt.tz_localize(None)
    return s.dt.to_period("Q").dt.end_time.dt.normalize()

def _build_one_factor(df, score_col, ret_col="return_after", qcut=0.3):
    """Quarterly EW long (top qcut) - short (bottom qcut) on score_col."""
    use = df[["date","ISIN",score_col,ret_col]].dropna().copy()
    use["qdate"] = _q_end(use["date"])

    cs = (use.groupby(["qdate","ISIN"], as_index=False)
               .agg({score_col:"mean", ret_col:"mean"}))

    rows = []
    for qd, g in cs.groupby("qdate"):
        g = g.copy()
        g["rk"] = g[score_col].rank(pct=True, method="first")
        lo, hi = qcut, 1.0 - qcut
        long_ret  = g.loc[g["rk"] >= hi, ret_col].mean()
        short_ret = g.loc[g["rk"] <= lo, ret_col].mean()
        rows.append({"date": pd.to_datetime(qd), f"{score_col}_ls": long_ret - short_ret})

    return pd.DataFrame(rows).sort_values("date").reset_index(drop=True)

def build_quarterly_factors_from_xs(df, ret_col="return_after",
                                    use_cols=("MOMENTUM","VALUE","SIZE"),
                                    qcut=0.3):
    """Build simple Carhart-like factors from your cross-section, quarterly."""
    facs = []
    for c in use_cols:
        if c in df.columns:
            facs.append(
                _build_one_factor(df, c, ret_col=ret_col, qcut=qcut)
                .rename(columns={f"{c}_ls": c})
            )
    if not facs:
        raise ValueError("No factor columns found in df for factor construction.")

    F = facs[0]
    for t in facs[1:]:
        F = pd.merge(F, t, on="date", how="outer")

    F = F.sort_values("date").reset_index(drop=True)

    # simple 'MKT' proxy = EW average quarterly return
    mkt = (df[["date","ISIN",ret_col]]
             .assign(qdate=_q_end(df["date"]))
             .groupby(["qdate","ISIN"], as_index=False)[ret_col].mean()
             .groupby("qdate", as_index=False)[ret_col].mean()
             .rename(columns={"qdate":"date", ret_col:"MKT"}))

    F = pd.merge(F, mkt, on="date", how="left")

    cols = ["date"] + [c for c in ["MKT","MOMENTUM","VALUE","SIZE"] if c in F.columns]
    return F[cols].dropna()


In [30]:
# -----------------------------
# Long–short portfolio builder (per quarter)
# -----------------------------
def ls_portfolio_quarterly(
    df,
    mts_col,
    ret_col="return_after",
    quantile=0.10,
    flip=True,
    min_total=10,
    min_per_side=2,
    print_debug=False
):
    """
    Build a quarterly long–short portfolio from a given signal column.

    Parameters
    ----------
    df : DataFrame with [date, ISIN, mts_col, ret_col]
    mts_col : str, name of signal column (e.g., "MTS_roll")
    ret_col : str, name of return column (default "return_after")
    quantile : float, quantile threshold (0.10 → 10-10 portfolio)
    flip : bool, if True, reverse long/short assignment
    min_total : int, minimum total names per cross-section
    min_per_side : int, minimum names per side (long/short)
    print_debug : bool, whether to print diagnostics

    Returns
    -------
    port_df : DataFrame with [date, long_ret, short_ret, port_ret, long_n, short_n, total_n]
    skip_log : list of skipped quarters with reasons
    """
    use = df[["date","ISIN",mts_col,ret_col]].dropna().copy()
    use["date"] = pd.to_datetime(use["date"]).dt.tz_localize(None)
    use["qdate"] = use["date"].dt.to_period("Q").dt.end_time.dt.normalize()

    # collapse duplicates within quarter
    agg = (use.groupby(["qdate","ISIN"], as_index=False)
               .agg({mts_col:"mean", ret_col:"mean"}))

    rows, skip_log = [], []
    for qd, cs in agg.groupby("qdate"):
        n = len(cs)
        if n < min_total:
            skip_log.append((qd, f"too few names: total={n} < {min_total}"))
            continue

        cs = cs.copy()
        cs["rank_pct"] = cs[mts_col].rank(pct=True, method="first")
        lo_cut, hi_cut = quantile, 1.0 - quantile

        longs  = cs[cs["rank_pct"] >= hi_cut]
        shorts = cs[cs["rank_pct"] <= lo_cut]
        n_long, n_short = len(longs), len(shorts)

        if n_long < min_per_side or n_short < min_per_side:
            skip_log.append((qd, f"too few per side: long={n_long}, short={n_short}"))
            continue

        long_ret  = longs[ret_col].mean()
        short_ret = shorts[ret_col].mean()

        # if flip=True, invert sides
        if flip:
            spread = short_ret - long_ret
        else:
            spread = long_ret - short_ret

        rows.append({
            "date": pd.to_datetime(qd),
            "long_ret": long_ret,
            "short_ret": short_ret,
            "port_ret": spread,
            "long_n": n_long,
            "short_n": n_short,
            "total_n": n
        })

    port_df = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)

    if print_debug:
        print(f"\n{mts_col} portfolio ({int(quantile*100)}–{int(quantile*100)}{' FLIPPED' if flip else ''})")
        if not port_df.empty:
            print(port_df)
        else:
            print("No valid quarters formed.")
        if skip_log:
            print("Skipped quarters:", skip_log)

    return port_df, skip_log

In [31]:
df_mts = build_all_mts(df, lookback=6, min_obs=60)

run_all_mts_factor_eval(
    df_mts,
    mts_cols=[
        "MTS_roll","MTS_ewma","MTS_median",
        "MTS_w_var","MTS_w_freq",
        "MTS_pca","MTS_lasso",
        "MTS_kl","MTS_js","MTS_cosine","MTS_euclid","MTS_ham","MTS_costate"
    ],
    ret_col="return_after",
    flip=True,
    quantile=0.10,
    hac_lags=1
)

/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/d


MTS_roll (10-10% FLIPPED)
n=8, mean=-8.3724, stdev=23.5717, sharpe=-0.36, hit_rate=0.25, cum_return=-50.23
Alpha (annualized, bps): 13634.0 | t(alpha)=2.01 | p=0.045 | R²=1.00, Adj R²=1.00
Betas: MKT=-3.766, MOMENTUM=-10.472, VALUE=7.437, SIZE=15.579

MTS_ewma (10-10% FLIPPED)
n=8, mean=-8.3640, stdev=23.6175, sharpe=-0.35, hit_rate=0.38, cum_return=-60.55
Alpha (annualized, bps): 12447.0 | t(alpha)=1.84 | p=0.066 | R²=1.00, Adj R²=1.00
Betas: MKT=-3.873, MOMENTUM=-9.067, VALUE=8.775, SIZE=15.466

MTS_median (10-10% FLIPPED)
n=8, mean=-8.3510, stdev=23.6014, sharpe=-0.35, hit_rate=0.38, cum_return=-63.32
Alpha (annualized, bps): 11173.9 | t(alpha)=1.70 | p=0.089 | R²=1.00, Adj R²=1.00
Betas: MKT=-3.417, MOMENTUM=-9.312, VALUE=8.715, SIZE=15.391

MTS_w_var (10-10% FLIPPED)
n=8, mean=-0.0092, stdev=0.0591, sharpe=-0.16, hit_rate=0.38, cum_return=-0.08
Alpha (annualized, bps): 685.2 | t(alpha)=0.92 | p=0.356 | R²=0.32, Adj R²=-0.58
Betas: MKT=0.076, MOMENTUM=-0.598, VALUE=-0.621, SIZE=0.

### Combo strategy

In [39]:
def build_mts_combo(df, signals=("MTS_kl","MTS_js","MTS_cosine"), 
                    method="sharpe", lookback_q=4):
    """
    Build a combined MTS signal from multiple inputs.
    
    Parameters
    ----------
    df : DataFrame
        Must contain the signal columns.
    signals : tuple of str
        Names of MTS signal columns to combine.
    method : str
        'sharpe' | 'pca' | 'vote'
    lookback_q : int
        Number of quarters to use when computing rolling Sharpe (for sharpe method).
    """
    d = df.copy()

    # Standardize each signal cross-sectionally
    for col in signals:
        d[col+"_std"] = _cs_zscore(d[col], by=d["date"])

    if method == "sharpe":
        sharpe_weights = {}
        for col in signals:
            port, _ = ls_portfolio_quarterly(
                d, mts_col=col+"_std", ret_col="return_after", 
                quantile=0.10, flip=True
            )
            sharpe = port["port_ret"].mean() / (port["port_ret"].std(ddof=1)+1e-12)
            sharpe_weights[col+"_std"] = max(sharpe,0)  # only positive weights

        s = sum(sharpe_weights.values())
        for k in sharpe_weights:
            sharpe_weights[k] /= (s if s>0 else 1)

        d["MTS_combo"] = sum(w*d[k] for k,w in sharpe_weights.items())

    elif method == "pca":
        X = d[[c+"_std" for c in signals]].fillna(0).to_numpy()
        pca = PCA(n_components=1).fit(X)
        d["MTS_combo"] = pca.transform(X)[:,0]

    elif method == "vote":
        vote_cols = []
        for col in signals:
            bcol = col+"_bin"
            d[bcol] = np.sign(d[col+"_std"])
            vote_cols.append(bcol)
        d["MTS_combo"] = np.sign(d[vote_cols].sum(axis=1))

    else:
        raise ValueError(f"Unknown method: {method}")

    return d

In [40]:
df_mts = build_all_mts(df, lookback=6, min_obs=60)

# Try Sharpe-weighted
df_mts1 = build_mts_combo(df_mts, signals=("MTS_kl","MTS_js","MTS_cosine"), method="sharpe")
port1, _ = ls_portfolio_quarterly(df_mts1, "MTS_combo", "return_after", flip=True)
print("Sharpe-weighted combo:", perf_line(port1))

# Try PCA
df_mts2 = build_mts_combo(df_mts, signals=("MTS_kl","MTS_js","MTS_cosine"), method="pca")
port2, _ = ls_portfolio_quarterly(df_mts2, "MTS_combo", "return_after", flip=True)
print("PCA combo:", perf_line(port2))

# Try Voting
df_mts3 = build_mts_combo(df_mts, signals=("MTS_kl","MTS_js","MTS_cosine"), method="vote")
port3, _ = ls_portfolio_quarterly(df_mts3, "MTS_combo", "return_after", flip=True)
print("Voting combo:", perf_line(port3))

/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/d

Sharpe-weighted combo: n=8, mean=0.0107, stdev=0.0584, sharpe=0.18, hit_rate=0.38, cum_return=0.08
PCA combo: n=8, mean=0.0179, stdev=0.1453, sharpe=0.12, hit_rate=0.12, cum_return=0.08
Voting combo: n=8, mean=0.0288, stdev=0.1132, sharpe=0.25, hit_rate=0.50, cum_return=0.21


/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T


In [41]:
def test_mts_combos(df, combo_list, methods=("sharpe","pca","vote"),
                    ret_col="return_after", flip=True, quantile=0.10):
    """
    Try different combos of MTS signals with different methods.
    combo_list = list of tuples (combo_name, [signals...])
    """
    results = []
    for name, signals in combo_list:
        for method in methods:
            try:
                d_combo = build_mts_combo(df, signals=signals, method=method)
                port, _ = ls_portfolio_quarterly(
                    d_combo, "MTS_combo", ret_col=ret_col,
                    flip=flip, quantile=quantile
                )
                results.append((name, method, perf_line(port)))
            except Exception as e:
                results.append((name, method, f"Error: {e}"))
    return results


# --------------------
# Define combos
# --------------------
combos = [
    ("KL+Cosine", ["MTS_kl","MTS_cosine"]),
    ("KL+Cosine+w_var", ["MTS_kl","MTS_cosine","MTS_w_var"]),
    ("KL+Cosine+JS", ["MTS_kl","MTS_cosine","MTS_js"]),
    ("Cosine+w_var", ["MTS_cosine","MTS_w_var"]),
    ("KL+JS+Cosine+w_var", ["MTS_kl","MTS_js","MTS_cosine","MTS_w_var"]),
]

# --------------------
# Run
# --------------------
df_mts = build_all_mts(df, lookback=6, min_obs=60)
results = test_mts_combos(df_mts, combos)

for name, method, perf in results:
    print(f"{name} ({method}): {perf}")


/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/d

KL+Cosine (sharpe): n=8, mean=0.0009, stdev=0.0433, sharpe=0.02, hit_rate=0.38, cum_return=0.00
KL+Cosine (pca): n=8, mean=-0.0141, stdev=0.0684, sharpe=-0.21, hit_rate=0.12, cum_return=-0.12
KL+Cosine (vote): n=8, mean=0.9138, stdev=2.1444, sharpe=0.43, hit_rate=0.50, cum_return=14.72
KL+Cosine+w_var (sharpe): n=7, mean=0.0137, stdev=0.0642, sharpe=0.21, hit_rate=0.71, cum_return=0.09
KL+Cosine+w_var (pca): n=8, mean=-0.0091, stdev=0.0812, sharpe=-0.11, hit_rate=0.12, cum_return=-0.09
KL+Cosine+w_var (vote): n=8, mean=0.7856, stdev=2.1629, sharpe=0.36, hit_rate=0.62, cum_return=6.98
KL+Cosine+JS (sharpe): n=8, mean=0.0107, stdev=0.0584, sharpe=0.18, hit_rate=0.38, cum_return=0.08
KL+Cosine+JS (pca): n=8, mean=0.0179, stdev=0.1453, sharpe=0.12, hit_rate=0.12, cum_return=0.08
KL+Cosine+JS (vote): n=8, mean=0.0288, stdev=0.1132, sharpe=0.25, hit_rate=0.50, cum_return=0.21
Cosine+w_var (sharpe): n=7, mean=0.0137, stdev=0.0642, sharpe=0.21, hit_rate=0.71, cum_return=0.09
Cosine+w_var (pca)

In [42]:
# --- Build the combo with PCA ---
df_mts = build_all_mts(df, lookback=6, min_obs=60)
df_mts = build_mts_combo(df_mts, signals=["MTS_cosine","MTS_w_var"], method="pca")

# --- Form long-short portfolio (10-10% flipped) ---
port, skip_log = ls_portfolio_quarterly(
    df_mts,
    mts_col="MTS_combo",
    ret_col="return_after",
    quantile=0.10,
    flip=True,
    print_debug=True   # <- will print portfolio details per quarter
)

# --- View results ---
print("\nQuarterly portfolio returns:")
print(port)

print("\nSkipped quarters (if any):", skip_log)

print("\nPerformance summary:")
print(perf_line(port))


/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/d


MTS_combo portfolio (10–10 FLIPPED)
        date  long_ret  short_ret  port_ret  long_n  short_n  total_n
0 2023-03-31 -0.006640  -0.003813  0.002827      39       38      387
1 2023-06-30  0.035651   0.083501  0.047850      41       40      404
2 2023-09-30  0.001394   0.022903  0.021509      41       40      401
3 2023-12-31  0.078574   0.193941  0.115367      41       40      402
4 2024-03-31 -0.020298   0.361313  0.381610      40       39      396
5 2024-06-30  0.079407   0.028239 -0.051169      41       40      404
6 2024-09-30  0.092868   0.089013 -0.003855      37       36      365
7 2024-12-31  0.000000   0.007534  0.007534       5        4       45

Quarterly portfolio returns:
        date  long_ret  short_ret  port_ret  long_n  short_n  total_n
0 2023-03-31 -0.006640  -0.003813  0.002827      39       38      387
1 2023-06-30  0.035651   0.083501  0.047850      41       40      404
2 2023-09-30  0.001394   0.022903  0.021509      41       40      401
3 2023-12-31  0.078574 

/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/komalniraula/myenv/lib/python3.13/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
